# Correlation of vertex betweenness with graph energy of vertex egocentric network

In this experiment we are considering the measure of energy dispersion and its correlation to betweeness of vertices. There are multiple graph energies proposed in the literature, but our primary interest lies in Graph, Laplacian and Randić energies. Within the experiment we calculate the energy of each vertex by means of their ego-network. Then we compare how betweenes of vertices is correlated with the energy of their egocentric networks.

Our second experiment aims at using machine learning to predict the betweenness of a vertex from only the local information contained in the egocentric network of that vertex. Computing of betweenness requires a very costly computation of all shortest paths in the network. Our approach allows to estimate betweennes using only local information.

Finally, in our third experiment, we perform transfer learning, training a model for predicting betweenness on one network, and using this model to predict betweenness of vertices in other networks.


## Graph energy

Graph energy of a graph is defined as $E_G(G) = \sum\limits_{i=1}^n |\mu_i|$, where $\mu_1, \ldots, \mu_n$ are the eigenvalues of the adjacency matrix $M_A$ (also known as the *spectrum* of the graph).

## Randić energy

Randić matrix of the graph $G=\left<V, E\right>$ is defined as:

$$
M_R(i,j)=
\begin{cases}
0 & \mathit{if} & i=j\\
\frac{1}{\sqrt{d_i d_j}} & \mathit{if} & (i,j) \in E\\
0 & \mathit{if} & (i,j) \notin E
\end{cases}
$$

Randić energy of a graph is defined as $E_R(G) = \sum\limits_{i=1}^n |\rho_i|$, where $\rho_1, \ldots, \rho_n$ are the eigenvalues of the Randić matrix $M_R$.

## Laplacian energy

Laplacian matrix of the graph $G=\left<V, E\right>$ is defined as:

$$
M_L(i,j)=
\begin{cases}
d_i & \mathit{if} & i=j\\
-1 & \mathit{if} & (i,j) \in E\\
0 & \mathit{otherwise}
\end{cases}
$$

Laplacian energy of a graph is defined as $E_L(G) =  \sum\limits_{i=1}^n |\lambda_i - \frac{2m}{n}|$, where $\lambda_1, \ldots, \lambda_n$ are the eigenvalues of the Laplacian matrix $M_L$, $n$ is the number of vertices and $m$ is the number of edges in the graph $G$.

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tqdm import tqdm_notebook as tqdm

from ggplot import *

import pandas as pd
import numpy as np
import networkx as nx
import scipy, scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing
import logging
import requests
import itertools

In [3]:
import sys
sys.path.append("..")

import network_energy as ne

In [4]:
def chunks(lst, n):
    """
    Divide a list of vertices `lst` into chunks consisting of `n` vertices
    
    Tests:
    >>> list(chunks([1,2,3,4,5,6], 2))
    [(1, 2), (3, 4), (5, 6)]

    >>> list(chunks([1,2,3,4,5,6], 4))
    [(1, 2, 3, 4), (5, 6)]

    >>> list(chunks([], 2))
    []

    """
    _lst = iter(lst)
    while 1:
        x = tuple(itertools.islice(_lst, n))
        if not x:
            return
        yield x
        
def normalize_df_column(df_column):
    """
    Normalize a dataframe column to the range [0,1]
    
    Tests:
    >>> normalize_df_column(pd.Series([1,2,3,4,5]))
    array([[0.  ],
           [0.25],
           [0.5 ],
           [0.75],
           [1.  ]])
    """
    x = df_column.values.astype(float)
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x.reshape(-1,1))
    
    return x_scaled

In [5]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

## Matrix energies for various topologies of small egocentric networks

Firstly, let us examine the relationship between the topology of a small egocentric network and its energies. We generate five different egocentric networks representing possible small scale configurations and compute all three types of matrix energies. The results are somehow surprising, graph energy tends to correlate with the degree of connectivity of the egocentric network, Randic energy remains practically constant, and Laplacian energy behaves unpredictably, receiving the maximum value for a custom topology. 

In [ ]:
g_custom = nx.star_graph(n=5)
g_custom.add_edge(1,2)
g_custom.add_edge(4,5)

graphs = [
    {'name': 'path', 'graph': nx.path_graph(n=3)},
    {'name': 'star', 'graph': nx.star_graph(n=5)},
    {'name': 'custom', 'graph': g_custom},
    {'name': 'wheel', 'graph': nx.wheel_graph(n=5)},
    {'name': 'complete', 'graph': nx.complete_graph(n=5)}
]

sns.set(rc={'figure.figsize': (15, 4)})
fig, ax = plt.subplots(5, 1)

df = pd.DataFrame(
    {'name': [ g['name'] for g in graphs],
     'graph energy': [ne.get_graph_energy(g['graph']) for g in graphs],
     'randic energy': [ne.get_randic_energy(g['graph']) for g in graphs],
     'laplacian energy': [ne.get_laplacian_energy(g['graph']) for g in graphs]
    }
)

plt.subplot(151)
nx.draw(graphs[0]['graph'], node_color=['r','y','r'])
plt.title(graphs[0]['name'])

plt.subplot(152)
nx.draw(graphs[1]['graph'], node_color=['y','r', 'r', 'r', 'r', 'r'])
plt.title(graphs[1]['name'])

plt.subplot(153)
nx.draw(graphs[2]['graph'], node_color=['y','r','r', 'r', 'r', 'r'])
plt.title(graphs[2]['name'])

plt.subplot(154)
nx.draw(graphs[3]['graph'], node_color=['y','r','r', 'r', 'r'])
plt.title(graphs[3]['name'])

plt.subplot(155)
nx.draw(graphs[4]['graph'], node_color=['y','r','r', 'r', 'r'])
plt.title(graphs[4]['name'])

plt.show()

df[['name', 'graph energy', 'randic energy', 'laplacian energy']]

Next, we start with a star configuration of an egocentric network consisting of an ego and additional $n$ vertices, and we gradually add all remaining edges, until we form a full $K_5$ graph. For each intermediate graph we compute all its energies. We can clearly see that each of matrix energies is measuring a different "aspect" of the egocentric network:

* randic energy is maximized for topologies very close to the original star-like structure and diminishes as more and more edges are added to the egocentric network
* laplacian energy strongly resembles the entropy of adjacency matrix, being maximized half-way between the star structure and the clique structure of the egocentric network
* graph energy steadily grows as the density of the egocentric network increases.

In [ ]:
from itertools import combinations
from random import shuffle

g = nx.star_graph(n=25)

results = []

edges = list(combinations(range(1, len(g.nodes)), r=2))

# comment if you want to add edges in an ordered way
shuffle(edges)

for (idx, (i, j)) in enumerate(edges):
    results.append((idx, ne.get_graph_energy(g), ne.get_randic_energy(g),
                    ne.get_laplacian_energy(g)))

    g.add_edge(i, j)

df = pd.DataFrame(
    data=results,
    columns=[
        'complexity', 'graph energy', 'randic energy', 'laplacian energy'
    ])

df['graph energy'] = normalize_df_column(df['graph energy'])
df['randic energy'] = normalize_df_column(df['randic energy'])
df['laplacian energy'] = normalize_df_column(df['laplacian energy'])

dfm = pd.melt(
    df,
    value_vars=['graph energy', 'randic energy', 'laplacian energy'],
    id_vars='complexity')

sns.set(rc={'figure.figsize': (12, 8)})
sns.lineplot(data=dfm, x='complexity', y='value', hue='variable')

# Correlation of graph/Randić/Laplacian energy and betweenness in artificial networks

In this experiment we create several instances of networks generated by two popular models:

- Erdos-Renyi random network model
- Barabasi-Albert preferential attachment model

and for each network we modify its main generative parameter. For each network we collect detailed statistics on every node:

- its betweenness,
- its Randić energy,
- its Laplacian energy,
- and its graph energy.

We normalize these variables using MinMax scaling to the range of [0-1]. Finally, we group the data by network model and network model parameter used to generate a given network instance, and for each such combination we compute the correlation of 



In [6]:
num_nodes = 250

results = pd.DataFrame(columns=['node',
                                'betweenness',
                                'randic_energy',
                                'graph_energy',
                                'network_model',
                                'network_model_param'])

# iterate over different parameter settings
network_model_params = [p/100 for p in range(1, 10)]

# radius of egocentric networks for energy computation
radius = 1

# sizes of groups for SBM
sizes = [50, 50, 150]

# intra- and inter-densities of edges between groups in SBM


def pa(): return np.random.uniform(0, 0.1)


def pb(): return np.random.uniform(0.2, 0.3)


for p in tqdm(network_model_params):

    probs = np.matrix([[pb(), pa(), pa()],
                       [pa(), pb(), pa()],
                       [pa(), pa(), pb()]])

    # symmetrize the matrix
    probs = np.maximum(probs, probs.T)

    # generate random and power-law networks with n=500 and p=0.01, 0.02, ..., 0.10
    generators = {
        'random': nx.erdos_renyi_graph(n=num_nodes, p=p),
        'powerlaw': nx.powerlaw_cluster_graph(n=num_nodes, m=2, p=p),
        'sbm': nx.stochastic_block_model(sizes=sizes, p=np.array(probs))
    }

    for generator in generators.keys():

        G = generators[generator]

        be = nx.betweenness_centrality(G, k=None)
        re = ne.randic_centrality(G, radius=radius)
        ge = ne.graph_energy_centrality(G, radius=radius)
        le = ne.laplacian_centrality(G, radius=radius)

        _dict = {
            'node': list(G.nodes),
            'betweenness': list(be.values()),
            'randic_energy': list(re.values()),
            'graph_energy': list(ge.values()),
            'laplacian_energy': list(le.values()),
            'network_model': [generator] * G.number_of_nodes(),
            'network_model_param': [p] * G.number_of_nodes()
        }

        _result = pd.DataFrame.from_dict(_dict)

        results = pd.concat([results, _result], axis=0)

In [7]:
# normalize columns to the range [0,1]

results.betweenness = normalize_df_column(results.betweenness)
results.randic_energy = normalize_df_column(results.randic_energy)
results.graph_energy = normalize_df_column(results.graph_energy)
results.laplacian_energy = normalize_df_column(results.laplacian_energy)

results[['betweenness','randic_energy','graph_energy','laplacian_energy']].describe()

,betweenness,randic_energy,graph_energy,laplacian_energy
count,6750.000000,6750.000000,6750.000000,6750.000000
mean,0.014029,0.340769,0.146380,0.137020
std,0.036699,0.228959,0.187477,0.175795
min,0.000000,0.000000,0.000000,0.000000
25%,0.003961,0.120695,0.016222,0.010233
50%,0.007575,0.235416,0.049650,0.056717
75%,0.012679,0.570438,0.201089,0.183017
max,1.000000,1.000000,1.000000,1.000000


In [8]:
# what is the correlation between betweenness and randić energy for different models?
_results = results.groupby(
    ['network_model_param',
     'network_model'])[['betweenness',
                        'randic_energy', 
                        'graph_energy',
                        'laplacian_energy']].corr().reset_index()

# extract correlations for the two models
powerlaw_idx = _results['network_model'] == 'powerlaw'
random_idx = _results['network_model'] == 'random'
sbm_idx = _results['network_model'] == 'sbm'

powerlaw_corr = _results[powerlaw_idx]['betweenness'].tolist()
random_corr = _results[random_idx]['betweenness'].tolist()
sbm_corr = _results[sbm_idx]['betweenness'].tolist()

# get additional columns with network model parameter and type of energy
network_model_param = _results[random_idx]['network_model_param'].tolist()
energy_idx = _results[random_idx]['level_2'].tolist()

correlations = pd.DataFrame({'p': network_model_param, 
                             'energy': energy_idx,
                             'powerlaw': powerlaw_corr, 
                             'random': random_corr,
                             'sbm': sbm_corr})

# melt the DataFrame to a format more suitable for drawing
correlations_mlt = pd.melt(correlations[correlations['energy'] != 'betweenness'], 
                id_vars=['p','energy'], 
                value_vars=['powerlaw','random','sbm'], 
                var_name='network_model', 
                value_name='correlation')

In [ ]:
g = sns.FacetGrid(correlations_mlt, col='energy', hue='network_model', height=8)
g.map(sns.lineplot, 'p', 'correlation')
g.add_legend()

# Visualization of graph energies centrality

We visualize Randić, Laplacian and graph energy for the well-known Zachary karate club network. The energy of each vertex is denoted by its color and size. As can be seen, the correlation between these centralities is very high and they all provide similar information. Also, one can see that these centralities have strong preference to nodes that are either central to their cluster, or lying in between clusters.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

norm = lambda lst: minmax.fit_transform(np.asarray(lst, dtype=np.float32).
                                        reshape(-1,1)).reshape(-1).tolist()

g = nx.karate_club_graph()

re = list(ne.randic_centrality(g).values())
re = norm(re)

le = list(ne.laplacian_centrality(g).values())
le = norm(le)

ge = list(ne.graph_energy_centrality(g).values())
ge = norm(ge)

sns.set(rc={'figure.figsize':(11.7,8.27)})

fig, ax = plt.subplots(3,1)

plt.subplot(131)
options = { 
    'node_color': [d * 1 for d in ge], 
    'node_size': [d * 1000 for d in ge], 
    'cmap': plt.cm.Greens,
    'edge_color': 'gray' 
}
nx.draw_kamada_kawai(g, **options)
plt.title('Graph energy')

plt.subplot(132)
options = { 
    'node_color': [d * 1 for d in le], 
    'node_size': [d * 1000 for d in le], 
    'cmap': plt.cm.Oranges,
    'edge_color': 'gray' 
}
nx.draw_kamada_kawai(g, **options)
plt.title('Laplacian energy')

plt.subplot(133)
options = { 
    'node_color': [d * 1 for d in re], 
    'node_size': [d * 1000 for d in re], 
    'cmap': plt.cm.Purples,
    'edge_color': 'gray'
}
nx.draw_kamada_kawai(g, **options)
plt.title('Randić energy')

plt.show()

# Regression modeling in synthetic networks

As the next step we fit regression models and compute their accuracy in terms of

  * Pearson, Spearman, and Kendall correlations
  * mean absolute error
  * mean squarred error
  
In the experiment we fit various regressors:

- simple linear model
- random forest regressor
- gradient boosting regressor

We collect the data from generative network models, and for each vertex we note its betweenness and its energies. Then, we fit these three regressors for networks generated for a particular value of the generative attribute.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr, kendalltau, rankdata

regression_models = {
    'linear model': LinearRegression(), 
    'random forest': RandomForestRegressor(), 
    'gradient boosting': GradientBoostingRegressor()
}

prediction_results = []

for network_model in results['network_model'].unique():
    for network_model_param in network_model_params:
        
        model_idx = results['network_model'] == network_model
        param_idx = results['network_model_param'] == network_model_param
        
        df = results[ model_idx & param_idx]
        
        y = df['betweenness'].values
        X = df[['randic_energy','graph_energy','laplacian_energy']].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        
        for regression_model in regression_models:
            
            l_model = regression_models[regression_model]
            
            l_model.fit(X=X_train, y=y_train)
            y_pred = l_model.predict(X_test)

            y_ranked = rankdata(y_test, method='ordinal') 
            y_pred_ranked = rankdata(y_pred, method='ordinal')

            _dict = {
                'regression_model': regression_model,
                'network_model': network_model,
                'network_model_param': network_model_param,
                'mae': mean_absolute_error(y_true=y_test, y_pred=y_pred),
                'mse': mean_squared_error(y_true=y_test, y_pred=y_pred),
                'r2': r2_score(y_test, y_pred),
                'pearson': pearsonr(y_test, y_pred)[0],
                'spearman': spearmanr(y_test, y_pred, axis=0, nan_policy='propagate')[0],
                'kendall': kendalltau(y_ranked, y_pred_ranked, initial_lexsort=None, nan_policy='propagate')[0]
            }

            prediction_results.append(_dict)

# convert the list of dicts into a DataFrame
prediction_results = pd.DataFrame(prediction_results, 
             columns=['regression_model', 
                      'network_model',
                      'network_model_param', 
                      'mae', 
                      'mse', 
                      'r2', 
                      'pearson', 
                      'spearman', 
                      'kendall'])

# melt DataFrame to transform it into tidy format
prediction_results_mlt = pd.melt(prediction_results,
                             id_vars=['regression_model', 'network_model','network_model_param'], 
                             var_name='measure',
                             value_name='value')

## Visualization of the relationship between betweenness and energies in synthetic networks

The following figures show the linear model predicting vertex betweenness based on graph energy, Randić energy, and Laplacian energy for random and powerlaw networks for a fixed value of the generative parameter. The value of the parameter is drawn randomly from the entire domain of the parameter.

In [ ]:
network_model = 'random'
network_model_param = np.random.choice(network_model_params)

sns.jointplot(x='randic_energy',
              y='betweenness', 
              data=results[(results['network_model'] == network_model) 
                           & (results['network_model_param'] == network_model_param)], 
              kind='reg', 
              color="xkcd:sky blue")

sns.jointplot(x='graph_energy', 
              y='betweenness',
              data=results[(results['network_model'] == network_model) 
                           & (results['network_model_param'] == network_model_param)], 
              kind="reg", 
              color="xkcd:strawberry")

sns.jointplot(x='laplacian_energy', 
              y='betweenness',
              data=results[(results['network_model'] == network_model) 
                           & (results['network_model_param'] == network_model_param)], 
              kind="reg", 
              color="xkcd:blue gray")


In [ ]:
network_model = 'powerlaw'
network_model_param = np.random.choice(network_model_params)

sns.jointplot(x='randic_energy', 
              y='betweenness', 
              data=results[(results['network_model'] == network_model)
                          & (results['network_model_param'] == network_model_param)], 
              kind='reg', 
              color="xkcd:medium blue")

sns.jointplot(x='graph_energy', 
              y='betweenness',
              data=results[(results['network_model'] == network_model)
                          & (results['network_model_param'] == network_model_param)], 
              kind="reg", 
              color="xkcd:pumpkin orange")

sns.jointplot(x='laplacian_energy', 
              y='betweenness',
              data=results[(results['network_model'] == network_model)
                          & (results['network_model_param'] == network_model_param)], 
              kind="reg", 
              color="xkcd:moss green")

Below we present correlation measures (Pearson, Spearman, Kendall) which describe the relationship of betweenness and vertex energies (Randic, Laplacian and Graph). We randomly select the value of the network generator parameter and repeat the correlation computation, averaging the results over 1000 runs.

In [ ]:
result = []

number_of_runs = 1000


for network_model in results.network_model.unique():
    for run in tqdm(range(number_of_runs)):
        
        network_model_param = np.random.choice(network_model_params)

        network_model_idx = results['network_model'] == network_model
        network_model_param_idx = results['network_model_param'] == network_model_param

        for method in ['kendall','spearman','pearson']:

            _corr = results[network_model_idx & network_model_param_idx][
                ['betweenness',
                 'randic_energy',
                 'graph_energy',
                 'laplacian_energy']].corr(method=method)
            
            result.append((network_model, 
                           network_model_param, 
                           method, 
                           _corr.betweenness['randic_energy'], 
                           _corr.betweenness['graph_energy'],
                           _corr.betweenness['laplacian_energy']))
        
correlations = pd.DataFrame(result, 
                            columns=['network_model', 
                                     'network_model_param',
                                     'method', 
                                     'BRC', 
                                     'BGC',
                                     'BLC'])

In [ ]:
correlations[['network_model','method','BRC','BGC','BLC']].groupby(['network_model','method']).mean()

In [ ]:
# melt DataFrame to transform it into tidy format
correlations_mlt = pd.melt(correlations,
                             id_vars=['network_model','network_model_param', 'method'], 
                             var_name='measure',
                             value_name='correlation')

In [ ]:
sns.reset_defaults()

g = sns.FacetGrid(correlations_mlt, col='network_model', row='measure', hue='method', height=5, sharey=False)
g.map(sns.lineplot, 'network_model_param', 'correlation')
g.add_legend()

# Empirical networks

In this experiment we are trying to verify if the relationship between vertex energies and vertex betweenness holds also in empirical networks. We download a set of networks from the Koblenz network repository and compute the basic statistics of these networks.

In [ ]:
from bs4 import BeautifulSoup
import requests
import wget
import tarfile
import os
import shutil
import time

In [ ]:
def read_avalilable_datasets_konect():
    base_url = "http://konect.uni-koblenz.de/downloads/"
    response = requests.get(base_url)
    
    if response.status_code != 200:
        print("An error occurred while getting data.")
    else:
        html = response.content
        soup = BeautifulSoup(html, "html5lib")
        
        table_html = soup.find(id='sort1')
        
        thead_html = table_html.find('thead')
        tbody_html = table_html.find('tbody')
         
        column_names=[row.text for row in thead_html.findAll('td')]
        rows = tbody_html.findAll('tr')
        values=[[cell.get('href') for cell in value('a') if 'tsv' in cell.get('href')] for value in rows]
        return [val[0].replace('.tar.bz2','').replace('tsv/','') for val in values]
        
def download_tsv_dataset_konect(network_name):
    assert (network_name in read_avalilable_datasets_konect()),"No network named: '"+network_name+"' found in Konect!"
    
    tsv_file = 'http://konect.uni-koblenz.de/downloads/tsv/'+network_name+'.tar.bz2'
    output_file=network_name+'.tar.bz2'
    file_name = wget.download(tsv_file, out=output_file)
    if os.path.exists(output_file):
        shutil.move(file_name,output_file)
    
    return output_file
    
def unpack_tar_bz2_file(file_name):
    tar = tarfile.open("./"+file_name, "r:bz2")
    output_dir="./network_"+file_name.replace('.tar.bz2','')+"/"
    tar.extractall(output_dir)
    tar.close()
    return output_dir

def build_network_from_out_konect(network_name):
    file_name=download_tsv_dataset_konect(network_name=network_name)
    output_dir=unpack_tar_bz2_file(file_name)+network_name+"/"
    files = [file for file in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, file))]
    out_file = [file for file in files if 'out.' in file]
    assert (len(out_file)>0), 'No out. file in the directory.'
    
    #building network
    G=nx.read_adjlist(output_dir+out_file[0], comments='%')
    
    return G

In [ ]:
networks_names=[
 'moreno_beach',
 'moreno_bison',
 'moreno_blogs',
 'moreno_cattle',
 'moreno_crime',
 'moreno_health',
 'moreno_highschool',
 'moreno_innovation',
 'moreno_kangaroo',
 'moreno_lesmis',
 'moreno_mac',
 'moreno_names',
 'moreno_oz',
 'moreno_propro',
 'moreno_rhesus',
 'moreno_sampson',
 'moreno_seventh',
 'moreno_sheep',
 'moreno_taro',
 'moreno_train',
 'moreno_vdb',
 'moreno_zebra',
]

In [ ]:
networks=[]

for network_name in tqdm(networks_names):
    networks.append(build_network_from_out_konect(network_name))

## Compute basic network statistics

In [ ]:
network_stats = []

for (name, network) in zip(networks_names, networks):
    network_stats.append((name, 
                          network.number_of_nodes(), 
                          network.number_of_edges(),
                          np.round(network.number_of_edges()/network.number_of_nodes(), 2)))
    
network_statistics_df = pd.DataFrame(network_stats, columns=['network name', 
                                          'number of vertices', 
                                          'number of edges', 
                                          'average degree'])

# print(network_statistics_df.to_latex(index=False))

In [ ]:
network_statistics_df

## Show the density ratio where our method beats SOTA

Our initial analysis points to a certain area of the (#vertices, avg.degree) parameter space, where our method of betweenness estimation should perform better than the best currently known exact algorithm of betweenness estimation. 

In [ ]:
x = range(1, network_statistics_df['number of vertices'].max())
y1 = np.power(x * np.log(x), 1 / 3)
y2 = np.power(x, 2 / 3)

df = pd.DataFrame({
    'num vertices': x,
    'best limit': y1,
    'naive limit': y2
}).melt(
    id_vars=['num vertices'], var_name='average degree limit')

sns.set(rc={'figure.figsize': (11.7, 8.27)})

figure = sns.lineplot(
    data=df, x='num vertices', y='value', hue='average degree limit')

plt.xlabel('number of vertices')
plt.ylabel('average vertex degree')

ax = sns.regplot(
    x=network_statistics_df['number of vertices'],
    y=network_statistics_df['average degree'],
    scatter=True,
    fit_reg=False,
    marker='o',
    scatter_kws={"s": 50})
# the "s" key in `scatter_kws` modifies the size of the marker

network_data_labels = network_statistics_df[
    network_statistics_df['number of vertices'] > 200]

[
    ax.text(p[0] + 2, p[1] + 3, p[2])
    for p in zip(network_data_labels['number of vertices'],
                 network_data_labels['average degree'],
                 network_data_labels['network name'])
]

plt.show(figure)

## Compute the correlation of betweenness and egocentric energies

Firstly we calculate betweenness and energy measures for each vertex

In [ ]:
real_data_measures = pd.DataFrame(columns=['node', 'value_type','value','network'])

for i in tqdm(range(len(networks))):
    G = networks[i]
    
    be = nx.betweenness_centrality(G, k=None)
    tmp_df = pd.DataFrame({'node': [i[0] for i in be.items()],
                         'value_type': ['betweenness' for i in be.items()],
                         'value': [i[1] for i in be.items()],
                         'network': [networks_names[i] for j in be.items()]
                        })
    tmp_df['value'] = normalize_df_column(tmp_df['value'])
    real_data_measures = pd.concat([real_data_measures,tmp_df])
        
        
    re = ne.randic_centrality(G)
    tmp_df = pd.DataFrame({'node': [i[0] for i in re.items()],
                         'value_type': ['randic' for i in re.items()],
                         'value': [i[1] for i in re.items()],
                         'network': [networks_names[i] for j in be.items()]
                        })
    tmp_df['value'] = normalize_df_column(tmp_df['value'])
    real_data_measures = pd.concat([real_data_measures, tmp_df])

    ge = ne.graph_energy_centrality(G)
    tmp_df = pd.DataFrame({'node': [i[0] for i in ge.items()],
                         'value_type': ['graph' for i in ge.items()],
                         'value': [i[1] for i in ge.items()],
                         'network': [networks_names[i] for j in be.items()]
                        })
    tmp_df['value'] = normalize_df_column(tmp_df['value'])
    real_data_measures = pd.concat([real_data_measures, tmp_df])
    
    le = ne.laplacian_centrality(G)
    tmp_df = pd.DataFrame({'node': [i[0] for i in le.items()],
                         'value_type': ['laplacian' for i in le.items()],
                         'value': [i[1] for i in le.items()],
                         'network': [networks_names[i] for j in be.items()]
                        })
    tmp_df['value'] = normalize_df_column(tmp_df['value'])
    real_data_measures = pd.concat([real_data_measures, tmp_df])

In [ ]:
real_data_measures.to_pickle('./real_networks_calulated_betweenness_and_energy.pickle')

In [ ]:
def evaluate_results(net, y, y_pred):
    
    real_prediction_results = pd.DataFrame(columns=['network', 'error_type', 'error_value'])
    
    mae = sklearn.metrics.mean_absolute_error(y_true=y, y_pred=y_pred)
    tmp_df = pd.DataFrame({'network': [net],
                         'error_type': ['MAE'], 
                         'error_value': [mae]
                        })
    real_prediction_results = pd.concat([real_prediction_results, tmp_df])

    mse = sklearn.metrics.mean_squared_error(y_true=y, y_pred=y_pred)
    tmp_df = pd.DataFrame({'network': [net],
                         'error_type': ['MSE'], 
                         'error_value': [mse]
                        })
    real_prediction_results = pd.concat([real_prediction_results, tmp_df])

    pearson,_ = scipy.stats.pearsonr(x=y,y=y_pred)
    tmp_df = pd.DataFrame({'network': [net],
                         'error_type': ['pearson'], 
                         'error_value': [pearson]
                        })
    real_prediction_results = pd.concat([real_prediction_results, tmp_df])

    spearman,_ = scipy.stats.spearmanr(a=y,b=y_pred, axis=0, nan_policy='propagate')
    tmp_df = pd.DataFrame({'network': [net],
                         'error_type': ['spearman'], 
                         'error_value': [spearman]
                        })
    real_prediction_results = pd.concat([real_prediction_results, tmp_df])


    y_ranked = scipy.stats.rankdata(y, method='ordinal') #może metoda average
    y_pred_ranked = scipy.stats.rankdata(y_pred, method='ordinal')

    kendall,_ = scipy.stats.kendalltau(x=y_ranked,y=y_pred_ranked, initial_lexsort=None, nan_policy='propagate')
    tmp_df = pd.DataFrame({'network': [net],
                         'error_type': ['kendall'], 
                         'error_value': [kendall]
                        })
    real_prediction_results = pd.concat([real_prediction_results, tmp_df])    
    
    return real_prediction_results

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

real_prediction_results = pd.DataFrame(columns=['network', 'error_type', 'error_value'])
models = []

real_data_measures = pd.read_pickle('./real_networks_calulated_betweenness_and_energy.pickle')


for net in tqdm(real_data_measures['network'].unique()):

        l_model = KNeighborsRegressor(n_neighbors=10, weights='distance')
        l_model = RandomForestRegressor()
    
        y = real_data_measures[(real_data_measures['network']==net) & 
                  (real_data_measures['value_type']=='betweenness')
                 ]['value'].values

        X = real_data_measures[(real_data_measures['network']==net) & 
                  (real_data_measures['value_type']=='graph')
                 ]['value'].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        l_model.fit(X_train.reshape(-1, 1), y_train)

        models.append(l_model)
        
        y_pred=l_model.predict(X_test.reshape(-1, 1))
        
        real_prediction_results=pd.concat([real_prediction_results,evaluate_results(net,y_test,y_pred)])
        
        
real_prediction_results.to_pickle('./real_prediction_results_errors.pickle')     

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.2)

real_prediction_results=pd.read_pickle('./real_prediction_results_errors.pickle') 

f, axs = plt.subplots(5, 1, figsize=(10, 28), sharex=True, sharey=False)

for error_type, ax in zip(real_prediction_results['error_type'].unique(),axs):
    
    x = real_prediction_results[real_prediction_results['error_type']==error_type]['network'].values
    y = real_prediction_results[real_prediction_results['error_type']==error_type]['error_value'].values
    
    colors = {'MAE': 'xkcd:salmon',
              'MSE': 'xkcd:pea green', 
              'pearson': 'xkcd:sky blue', 
              'spearman': 'xkcd:goldenrod', 
              'kendall': 'xkcd:moss green'}
    
    plt.ylim([0,1])    
    ax.set_xticklabels(x,rotation='vertical')
    ax.set_title(error_type, fontsize=18)
    ax.set(xlabel='Network', ylabel='Value')
    sns.barplot(x, y, ax=ax, color=colors[error_type])
    
sns.despine(bottom=True)
plt.tight_layout(h_pad=3)


In [ ]:
df = pd.pivot_table(real_prediction_results.round(2), 
                    index='network', 
                    columns='error_type', 
                    values='error_value').rename_axis(None, axis=1)
# print(df.to_latex())
df

# Transfer learning

In this experiment we check if it is possible to train the regression model on one network and then transfer the same model to other networks. In other words, if the relationship between vertex betweenness and vertex energy is universal, such transfer learning should be possible.

In [ ]:
real_data_measures = pd.read_pickle('./real_networks_calulated_betweenness_and_energy.pickle')

transfer_real_prediction_results = pd.DataFrame(columns=['network', 'error_type', 'error_value', 'source_network'])

raw_results = pd.DataFrame(columns=['network', 'source_network', 'y', 'y_pred'])


for i in tqdm(range(len(models))):
    for net in (real_data_measures['network'].unique()):
        y=real_data_measures[(real_data_measures['network']==net) & 
                  (real_data_measures['value_type']=='betweenness')
                 ]['value'].values

        X=real_data_measures[(real_data_measures['network']==net) & 
                  (real_data_measures['value_type']=='graph')
                 ]['value'].values
        
        y_pred = models[i].predict(X.reshape(-1, 1))
        
        eval_res = evaluate_results(net,y,y_pred)
        eval_res['source_network'] = networks_names[i]
        
        raw_results = pd.concat([raw_results, pd.DataFrame.from_dict({'network': net, 
                                                                      'source_network': networks_names[i], 
                                                                      'y': y, 
                                                                      'y_pred':y_pred})])
        
        transfer_real_prediction_results = pd.concat([transfer_real_prediction_results,eval_res])

Lets draw a heatmap representing betweenness estimation errors and rank correlations for empirical networks.

In [ ]:
sns.set_style("whitegrid")
sns.set_palette('Oranges')
sns.set_context("notebook", font_scale=1.8)

for error_type in ['MAE', 'MSE']:
    
    error_idx = transfer_real_prediction_results['error_type']==error_type
    
    x = transfer_real_prediction_results[error_idx]['network'].unique()
    y = transfer_real_prediction_results[error_idx]['network'].unique()

    val = np.array(transfer_real_prediction_results[error_idx]['error_value'])
    val = val.reshape(len(x),len(y))

    plt.figure(figsize=(50,50))
    plt.title(error_type)
    
    to_draw=transfer_real_prediction_results[transfer_real_prediction_results['error_type']==error_type]
    
    ax = sns.heatmap(to_draw[['error_value', 
                              'source_network', 
                              'network']].pivot('source_network',
                                                'network',
                                                'error_value'), 
                     linewidth=0.5,
                     annot=True,
                     cmap='Oranges')
    ax.invert_yaxis()
    
    plt.show()
    

In [ ]:
for error_type in ['pearson', 'spearman', 'kendall']:
    
    error_idx = transfer_real_prediction_results['error_type']==error_type
    
    x = transfer_real_prediction_results[error_idx]['network'].unique()
    y = transfer_real_prediction_results[error_idx]['network'].unique()

    val = np.array(transfer_real_prediction_results[error_idx]['error_value'])
    val = val.reshape(len(x),len(y))

    plt.figure(figsize=(50,50))
    plt.title(error_type)
    
    to_draw=transfer_real_prediction_results[error_idx]
    
    ax = sns.heatmap(to_draw[['error_value', 
                              'source_network', 
                              'network']].pivot('source_network',
                                                'network',
                                                'error_value'), 
                     linewidth=0.5,
                     annot=True,
                     cmap='Oranges')
    ax.invert_yaxis()
    
    plt.show()
    